# OPERATION PROGRAMS
1. Setup
2. Place fresh naked orders

# 1. Setup

In [1]:
MARKET = 'SNP'

In [2]:
import sys
import pathlib
import numpy as np
import pandas as pd
import yaml
import asyncio

from ib_insync import IB, util, Option, MarketOrder, Contract
from typing import Callable, Coroutine, Union

In [3]:
# Specific to Jupyter. Will be ignored in IDE / command-lines
import IPython as ipy
if ipy.get_ipython().__class__.__name__ == 'ZMQInteractiveShell':
    import nest_asyncio
    nest_asyncio.apply()
    util.startLoop()
    pd.options.display.max_columns = None
    
    THIS_FOLDER = '' # Dummy for jupyter notebook's current folder 

In [4]:
# Get capability to import programs from `asyncib` folder
cwd = pathlib.Path.cwd() # working directory from where python was initiated
DATAPATH = cwd.joinpath('data', MARKET.lower()) # path to store data files
LOGFILE = DATAPATH.joinpath('temp.log') # path to store log files

IBPATH = cwd.parent.parent.joinpath('asyncib') # where ib programs are stored

# append IBPATH to import programs.
if str(IBPATH) not in sys.path:  # Convert it to string!
    sys.path.append(str(IBPATH))
    
IBDATAPATH = IBPATH.joinpath('data', MARKET.lower())

In [5]:
# Get the host, port, cid
from engine import Vars

ibp = Vars(MARKET.upper())  # IB Parameters from var.yml
HOST, PORT, CID = ibp.HOST, ibp.PORT, ibp.CID

In [6]:
# Get the pickle files
from os import listdir
fs = listdir(DATAPATH)

files = [f for f in fs if f[-4:] == '.pkl']
for f in files:
    exec(f"{f.split('.')[0]} = pd.read_pickle(DATAPATH.joinpath(f))")
np.sort(np.array(files))

array(['df_chains.pkl', 'df_fresh.pkl', 'df_ohlcs.pkl',
       'df_opt_margins.pkl', 'df_opt_prices.pkl', 'df_symlots.pkl',
       'df_und_margins.pkl', 'df_unds.pkl', 'dfrq.pkl', 'qopts.pkl'],
      dtype='<U18')

# 2. Placing `fresh` Naked Orders

In [26]:
# * IMPORTS
from ib_insync import LimitOrder
from fresh import get_fresh
from support import get_openorders

In [29]:
# * RUN FRESH
THIS_FOLDER = ''
ip = input("Do you want to run Fresh (Y/N)?")
if ip.upper() == "Y":
    df_fresh = get_fresh(MARKET)
else:
    df_fresh = pd.read_pickle(DATAPATH.joinpath(THIS_FOLDER, 'df_fresh.pkl'))

cols = ['symbol', 'strike', 'right', 'expiry', 'dte', 'conId', 'contract',
        'margin', 'bid', 'ask', 'iv', 'und_iv','price', 'lot', 'rom', 'sdMult', 'expRom', 'expPrice', 'qty']

df = df_fresh[df_fresh.price>0][cols].sort_values('rom', ascending=False)

Do you want to run Fresh (Y/N)? n


In [35]:
# * REMOVE OPEN ORDERS FROM FRESH df
df_openords = get_openorders(MARKET)
df = df[~df.conId.isin(df_openords.conId)].reset_index(drop=True)

# ... build the fresh SELL orders
contracts = df.contract.to_list()
orders = [LimitOrder(action='SELL', totalQuantity=abs(int(q)), lmtPrice=p) 
                        for q, p in zip(df.qty, df.expPrice)]

fresh_cos = [(c, o) for c, o in zip(contracts, orders)]

## Place `fresh` naked trades
* Make the following cell executable

## Clean up `fresh` trades

In [57]:
import os

fs = listdir(DATAPATH)

for f in fs:
    if f.split('.')[0] == 'df_fresh':
        try:
            os.remove(DATAPATH.joinpath(f))
        except OSError as e:
            print(f'Error: {e}')